In [47]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [50]:
index_im_fol = os.path.abspath("Images/index")
index_images = [os.path.join(index_im_fol, x) for x in os.listdir(index_im_fol)]

In [51]:
index_descriptors = []
index_keypoints = []
surf = cv2.xfeatures2d.SURF_create(10000)

In [52]:
def descrip():
    for image in index_images:
        img = cv2.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = surf.detectAndCompute(img,None)
        if des is not None:
            index_descriptors.extend(des)
            index_keypoints.extend(kp)

In [56]:
%lprun -f descrip descrip()

In [53]:
print(np.array(index_descriptors).shape)
print(len(index_keypoints))

(0,)
0
